In [2]:
# %% [BLOCCO 1] LIBRERIA FUNZIONI
# Qui definiamo gli strumenti matematici descritti nel paper.
import numpy as np
import networkx as nx
from metodi import get_google_matrix, power_method, print_ranking
import time


In [3]:
# %% [BLOCCO 2] GRAFO 1 (Figura 2.1 del paper)
# Ricostruzione manuale della matrice A.
# Regola: Se la pagina J linka a I, allora A[i,j] = 1 / (num link uscenti da J)

# COLONNA 1 (Pag 1): Linka a 2, 3, 4 -> 1/3 a ciascuno
# COLONNA 2 (Pag 2): Linka a 3, 4    -> 1/2 a ciascuno
# COLONNA 3 (Pag 3): Linka a 1       -> 1/1 a pag 1
# COLONNA 4 (Pag 4): Linka a 1, 3    -> 1/2 a ciascuno

#     P1   P2   P3   P4
A1 = np.array([
    [0.0, 0.0, 1.0, 0.5], # Verso Pagina 1
    [1/3, 0.0, 0.0, 0.0], # Verso Pagina 2
    [1/3, 0.5, 0.0, 0.5], # Verso Pagina 3
    [1/3, 0.5, 0.0, 0.0]  # Verso Pagina 4
])

print("Matrice A (Grafo 1):")
print(A1)

# Trasformiamo in Matrice Google M e calcoliamo
M1 = get_google_matrix(A1, m=0.15)
scores1, iters1 = power_method(M1)
print_ranking(scores1, "Risultati Grafo 1")

Matrice A (Grafo 1):
[[0.         0.         1.         0.5       ]
 [0.33333333 0.         0.         0.        ]
 [0.33333333 0.5        0.         0.5       ]
 [0.33333333 0.5        0.         0.        ]]

--- Risultati Grafo 1 ---
Pagina 1: 0.36815
Pagina 2: 0.14181
Pagina 3: 0.28796
Pagina 4: 0.20208


In [4]:
# %% [BLOCCO 3] GRAFO 2 (Figura 2.2 - Disconnesso)
# Due isole separate: {1,2} e {3,4,5}

#     P1   P2   P3   P4   P5
A2 = np.array([
    [0.0, 1.0, 0.0, 0.0, 0.0], # 1 riceve da 2
    [1.0, 0.0, 0.0, 0.0, 0.0], # 2 riceve da 1
    [0.0, 0.0, 0.0, 1.0, 0.5], # 3 riceve da 4 (CORRETTO: 1.0) e 5 (0.5)
    [0.0, 0.0, 1.0, 0.0, 0.5], # 4 riceve da 3 (1.0) e 5 (0.5)
    [0.0, 0.0, 0.0, 0.0, 0.0]  # 5 non riceve da nessuno
])

# Nota: La colonna 5 si divide tra riga 3 e riga 4. 
# La riga 5 è tutta 0 perché nessuno linka a 5.

M2 = get_google_matrix(A2, m=0.15)
scores2, iters2 = power_method(M2)
print_ranking(scores2, "Risultati Grafo 2 (Disconnesso)")



--- Risultati Grafo 2 (Disconnesso) ---
Pagina 1: 0.20000
Pagina 2: 0.20000
Pagina 3: 0.28500
Pagina 4: 0.28500
Pagina 5: 0.03000


In [5]:
# %% [BLOCCO 4] ESERCIZIO 4 (Dangling Node)
# Partiamo dal Grafo 1, ma RIMUOVIAMO il link da 3 a 1.
# La pagina 3 ora non linka a nessuno.

#     P1   P2   P3   P4
A_ex4 = np.array([
    [0.0, 0.0, 0.0, 0.5], # P3 -> P1 rimosso (era 1.0, ora 0.0)
    [1/3, 0.0, 0.0, 0.0], 
    [1/3, 0.5, 0.0, 0.5], 
    [1/3, 0.5, 0.0, 0.0] 
])
# Nota: La colonna 3 ora è TUTTA ZERI. È substocastica.

print("\n--- Analisi Esercizio 4 (Dangling Node) ---")
# Usiamo gli autovalori diretti (numpy.linalg.eig) perché il power method classico
# su matrici substocastiche tende a svanire a 0.
eigvals, eigvecs = np.linalg.eig(A_ex4)

# Prendiamo solo la parte reale e cerchiamo il massimo
real_eigvals = eigvals.real
idx_max = np.argmax(real_eigvals)
lambda_perron = real_eigvals[idx_max]
v_perron = eigvecs[:, idx_max].real

# Normalizziamo
if v_perron.sum() < 0: v_perron = -v_perron # Correggi segno se negativo
v_perron = v_perron / v_perron.sum()

print(f"Autovalore di Perron (lambda): {lambda_perron:.4f}")
print_ranking(v_perron, "Ranking Esercizio 4")


--- Analisi Esercizio 4 (Dangling Node) ---
Autovalore di Perron (lambda): 0.5614

--- Ranking Esercizio 4 ---
Pagina 1: 0.20665
Pagina 2: 0.12271
Pagina 3: 0.43865
Pagina 4: 0.23200


In [6]:
# %% [BLOCCO 5] ESERCIZIO 11 (Aggiunta Pagina 5)
# Modifichiamo il Grafo 1 aggiungendo P5.
# - P5 linka a P3.
# - P3 linka a P5 (oltre che a P1, come faceva prima).

#     P1   P2   P3   P4   P5
A_ex11 = np.array([
    # Col 3 modificata: prima dava 1.0 a P1, ora divide tra P1 e P5
    [0.0, 0.0, 0.5, 0.5, 0.0], # P1 riceve 0.5 da P3, 0.5 da P4
    [1/3, 0.0, 0.0, 0.0, 0.0], # P2 (invariato)
    [1/3, 0.5, 0.0, 0.5, 1.0], # P3 riceve da 1, 2, 4 E ORA DA 5 (1.0)
    [1/3, 0.5, 0.0, 0.0, 0.0], # P4 (invariato)
    [0.0, 0.0, 0.5, 0.0, 0.0]  # P5 riceve 0.5 da P3
])

print("\n--- Analisi Esercizio 11 (Pagina 5 aggiunta) ---")
print("Nuova Matrice A (5x5):")
print(A_ex11)

M_ex11 = get_google_matrix(A_ex11, m=0.15)
scores_ex11, iters_ex11 = power_method(M_ex11)
print_ranking(scores_ex11, "Risultati Esercizio 11")


--- Analisi Esercizio 11 (Pagina 5 aggiunta) ---
Nuova Matrice A (5x5):
[[0.         0.         0.5        0.5        0.        ]
 [0.33333333 0.         0.         0.         0.        ]
 [0.33333333 0.5        0.         0.5        1.        ]
 [0.33333333 0.5        0.         0.         0.        ]
 [0.         0.         0.5        0.         0.        ]]

--- Risultati Esercizio 11 ---
Pagina 1: 0.23714
Pagina 2: 0.09719
Pagina 3: 0.34889
Pagina 4: 0.13850
Pagina 5: 0.17828


In [5]:
class PageRankEngine:
    def __init__(self, damping_factor=0.85, tolerance=1e-9, max_iter=100):
        # FASE 1: IL TELAIO
        # Impostazioni matematiche
        self.d = damping_factor      # 85% di probabilità di seguire i link
        self.m = 1.0 - self.d        # 15% di probabilità di teletrasporto
        self.tol = tolerance         # Quando smettere (precisione)
        self.max_iter = max_iter     # Limite di sicurezza giri
        
        # Strutture Dati (Memoria del Grafo)
        self.urls = {}           # Mappa ID -> Nome Sito
        self.links = {}          # Mappa ID -> Lista di chi linko (es. 1 -> [2, 3])
        self.out_degree = {}     # Mappa ID -> Quanti link ho in uscita
        self.num_nodes = 0       # Contatore totale nodi
        self.node_list = []      # Elenco semplice degli ID [1, 2, 3...]

    def load_data(self, filename):
        #FASE 2: IL CARBURANTE - Legge il file e riempie le strutture
        print(f"--- Caricamento file: {filename} ---")
        max_id = 0
        
        try:
            with open(filename, 'r', encoding='latin-1') as f:
                # Saltiamo la prima riga (spesso contiene solo totali inutili)
                first_line = f.readline()
                
                for line in f:
                    # Pulizia della riga
                    parts = line.strip().replace(',', ' ').split()
                    if not parts: continue
                    
                    # Riconoscimento: È una definizione di URL?
                    if len(parts) >= 2 and 'http' in parts[1]:
                        nid = int(parts[0])
                        url = parts[1]
                        self.urls[nid] = url
                        max_id = max(max_id, nid)
                        
                    # Riconoscimento: È un collegamento (Link)?
                    elif len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                        src = int(parts[0])
                        dst = int(parts[1])
                        max_id = max(max_id, src, dst)
                        
                        # Creiamo la lista se è la prima volta che vediamo 'src'
                        if src not in self.links: 
                            self.links[src] = []
                        self.links[src].append(dst)
                        
        except FileNotFoundError:
            print(f"ERRORE CRITICO: Non trovo il file '{filename}'.")
            return False

        # Finalizzazione caricamento
        self.num_nodes = max_id
        self.node_list = range(1, self.num_nodes + 1) # Creiamo la lista ID da 1 a N
        
        # Calcoliamo i gradi di uscita per TUTTI i nodi
        for i in self.node_list:
            if i in self.links:
                self.out_degree[i] = len(self.links[i])
            else:
                # Se non è in self.links, è un DANGLING NODE (Vicolo cieco)
                self.out_degree[i] = 0 
                self.links[i] = []
                
        print(f"Grafo pronto: {self.num_nodes} nodi.")
        return True

    def compute_rank(self):
        """FASE 3: IL MOTORE - L'algoritmo matematico"""
        print("\n--- Inizio Calcolo PageRank ---")
        start_time = time.time()
        
        N = self.num_nodes
        
        # Stato Iniziale: Democrazia (1/N a tutti)
        scores = {node: 1.0 / N for node in self.node_list}
        
        # Inizia il ciclo iterativo (Giorno dopo giorno...)
        for it in range(self.max_iter):
            new_scores = {node: 0.0 for node in self.node_list}
            
            # --- GESTIONE DANGLING NODES (Teoria Langville & Meyer) ---
            # 1. Calcoliamo quanti voti sono finiti in vicoli ciechi
            dangling_sum = 0.0
            for node in self.node_list:
                if self.out_degree[node] == 0:
                    dangling_sum += scores[node]
            
            # 2. Calcoliamo la "Paga Base" universale
            # (Tassa Teletrasporto + Voti Dangling Reciclati) / N
            base_val = (self.m + (self.d * dangling_sum)) / N
            
            # Diamo la paga base a tutti
            for node in self.node_list:
                new_scores[node] = base_val
                
            # --- FASE DI SPINTA (PUSH) ---
            # Chi ha link, distribuisce il proprio voto (tassato all'85%)
            for src in self.node_list:
                n_out = self.out_degree[src]
                if n_out > 0:
                    # Calcola la fetta per ogni amico
                    share = (scores[src] * self.d) / n_out
                    # Spedisci la fetta
                    for dst in self.links[src]:
                        if dst <= N: # Controllo di sicurezza
                            new_scores[dst] += share
                            
            # Controllo Convergenza: I punteggi sono cambiati rispetto a ieri?
            diff = sum(abs(new_scores[n] - scores[n]) for n in self.node_list)
            scores = new_scores # Aggiorniamo per il domani
            
            if diff < self.tol:
                print(f"Il sistema si è stabilizzato dopo {it+1} iterazioni.")
                break
            
        end_time = time.time()
        print(f"Tempo di calcolo: {end_time - start_time:.4f} secondi")
        return scores

    def print_top_n(self, scores, n=10):
        """FASE 4: IL CRUSCOTTO - Mostra i vincitori"""
        # Ordiniamo il dizionario dei punteggi
        ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\n=== CLASSIFICA FINALE (TOP {n}) ===")
        print(f"{'Pos':<4} | {'Score':<10} | {'ID':<5} | {'URL'}")
        print("-" * 80)
        
        for i in range(min(n, len(ranked))):
            node_id, score = ranked[i]
            url = self.urls.get(node_id, "N/A") # Recuperiamo il nome leggibile
            
            # Tagliamo URL troppo lunghi per bellezza
            if len(url) > 50: url = url[:47] + "..."
            
            print(f"{i+1:<4} | {score:.6f}   | {node_id:<5} | {url}")

# --- AVVIO ---
if __name__ == "__main__":
    # Creiamo il motore
    engine = PageRankEngine(damping_factor=0.85)
    
    # Carichiamo il file (Assicurati che il nome sia corretto!)
    filename = 'hollins (2).dat' 
    
    if engine.load_data(filename):
        # Calcoliamo
        final_scores = engine.compute_rank()
        # Stampiamo
        engine.print_top_n(final_scores, 10)

--- Caricamento file: hollins (2).dat ---
Grafo pronto: 6012 nodi.

--- Inizio Calcolo PageRank ---
Il sistema si è stabilizzato dopo 97 iterazioni.
Tempo di calcolo: 0.3940 secondi

=== CLASSIFICA FINALE (TOP 10) ===
Pos  | Score      | ID    | URL
--------------------------------------------------------------------------------
1    | 0.019879   | 2     | http://www.hollins.edu/
2    | 0.009288   | 37    | http://www.hollins.edu/admissions/visit/visit.htm
3    | 0.008610   | 38    | http://www.hollins.edu/about/about_tour.htm
4    | 0.008065   | 61    | http://www.hollins.edu/htdig/index.html
5    | 0.008027   | 52    | http://www.hollins.edu/admissions/info-request/...
6    | 0.007165   | 43    | http://www.hollins.edu/admissions/apply/apply.htm
7    | 0.006583   | 425   | http://www.hollins.edu/academics/library/resour...
8    | 0.005989   | 27    | http://www.hollins.edu/admissions/admissions.htm
9    | 0.005572   | 28    | http://www.hollins.edu/academics/academics.htm
10   | 0.00